In [8]:
import time
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs
import requests
import json

img_path = '/home/jovyan/matlida_platform_demo/brain_dataset/test_input_1.jpg'
img_raw = tf.image.decode_image(
    open(img_path, 'rb').read(), channels=3)

img = tf.expand_dims(img_raw, 0)
img = transform_images(img, 256)
print(img.shape)
img = img.numpy().tolist()

data = json.dumps({"inputs": img})

(1, 256, 256, 3)


In [9]:
model_serve_url = "input_your_endpoint"

res_svc = requests.post(model_serve_url, data)
print(res_svc)
class_names = [c.strip() for c in open('/home/jovyan/matlida_platform_demo/brain_dataset/class.txt').readlines()]
print(class_names)

<Response [200]>
['negative', 'positive']


In [14]:
output_data = json.loads(res_svc.text)
print(output_data['outputs'].keys())

boxes = output_data['outputs']['yolo_nms']
scores = output_data['outputs']['yolo_nms_1']
classes = output_data['outputs']['yolo_nms_2']
nums = output_data['outputs']['yolo_nms_3']

for i in range(nums[0]):
    print('\t{}, {}, {}'.format(class_names[int(classes[0][i])],scores[0][i],boxes[0][i]))
    


dict_keys(['yolo_nms_1', 'yolo_nms_2', 'yolo_nms_3', 'yolo_nms'])
	positive, 0.578840494, [0.31247583, 0.561915398, 0.454335183, 0.694901]


In [18]:
img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
img = draw_outputs(img, (np.array(boxes), scores, classes, nums), class_names)
cv2.imwrite('./serve_output.jpg', img)

draw_outputs
1
(100, 4)
[192 256]
(59, 143)
(87, 177)


True